# Information

Tên dự án: Image resize using seam carving

Thành viên:
  - 19127597 - Trần Khả Trí
  - 19127607 - Trần Nguyên Trung

## 1. Application description

### What is your chosen application?
Seam carving là một phương pháp thay đổi kích thước ảnh, ý tưởng của thuật toán là loại bỏ những điểm ảnh ít quan trọng trong hình ảnh và giữ lại những điểm ảnh quan trọng để hình ảnh đạt được kích thước cần thiết.
- Input: Ảnh có định dạng PNM, tên file kết quả, số seam cần xoá
- Output: Ảnh đã được xoá seam


In [1]:
WORKDIR = '/content/seam-carving'
!git clone https://github.com/tristran2k1/seam-carving.git $WORKDIR
%cd $WORKDIR

Cloning into '/content/seam-carving'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 73 (delta 26), reused 55 (delta 14), pack-reused 0
Unpacking objects: 100% (73/73), done.
/content/seam-carving


## 2. Sequential implementation

Phiên bản chạy trên host, các bước thực hiện gồm có:
- B1. Đọc ảnh
- Lặp lại:
  - B2. Chuyển ảnh thành grayscale
  - B3. Blur ảnh sử dụng Gaussian Blur
  - B4. Detect các điểm cạnh sử dụng Sobel Filter
  - B5. Xây dựng Energy Map từ các cạnh theo phương ngang và phương dọc tương ứng
  - B6. Từ năng lượng của các điểm ảnh, xâu chuỗi lại thành danh sách các đường (seam). Những đường có ít năng lượng là những đường có ít giá trị đóng góp cho nội dung hình ảnh.
  - B7. Loại bỏ các đường có năng lượng thấp.

In [2]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


In [3]:
!nvcc -arch=sm_{major}{minor} /content/seam-carving/src/host/host.cu -o host

In [4]:
!./host /content/seam-carving/assets/input.pnm /content/seam-carving/assets/output.pnm 300

Version CPU, 300 seams: 31463.185547 ms


### Does it need to speed up?
Ứng dụng cần được tăng tốc vì thuật toán seam carving có độ phức tạp lớn khi thực hiện tuần tự(các bước hầu hết có độ phức tạp $O(n^2)$)
- Chuyển ảnh thành grayscale tốn $O(n^2)$, các pixel thực hiện độc lập với nhau
- Các bước convolution (blur ảnh, tìm cạnh, xây dựng energy map) tốn $O(n^2)$ để thực hiện tích chập trên từng pixel ảnh 
- Bước quy hoạch động tìm seam có độ quan trọng nhỏ nhất hay xoá 1 seam đã được tìm thấy đều có độ phức tạp $O(n^2)$
- Xoá seams tốn $O(n^2)$, từng hàng thực hiện độc lập với nhau. Giống như một phép copy ma trận nhưng không tính những pixel thuộc về seams loại bỏ.

**$\Rightarrow$ Việc áp dụng tính toán song song trong bài toán này là rất phù hợp khi phần lớn các bước chính của thuật toán ta đều có thể song song hoá**

## 3. Parallel implementation

In [5]:
!nvcc -arch=sm_{major}{minor} /content/seam-carving/src/device_1/device_1.cu -o device_1

In [6]:
!./device_1 /content/seam-carving/assets/input.pnm /content/seam-carving/assets/output.pnm 300

Version GPU 1, 300 seams: 3122.424561 ms


## 4. Parallel implementation + optimization

## 5. Reflection
### What difficulties have you encountered?
**Trung**
- Gặp khó khăn khi chưa thành thạo với các syntax khi lập trình trên GPU.
- Việc tối ưu các version song song.
- Hạn chế về thời gian khiến việc thực hiện bị áp lực, chưa được chu đáo, cẩn thận.

**Trí**

### What have you learned?
**Trung**
- Teamwork, phối hợp với nhau để thực hiện đồ án.
- Cách làm việc chuyên nghiệp hơn, các thành viên sẽ có các buổi họp thảo luận và chia sẻ về phần của mình, có khó khăn gì sẽ được thành viên khác giúp đỡ, thời gian được phân chia rõ ràng cụ thể để thực hiện từng giai đoạn của đồ án.
- Tìm kiếm thông tin, tìm hiểu các phương pháp, các thuật toán giúp hoàn thiện đồ án. Việc tìm kiếm thông tin trên internet giúp em hiểu rõ hơn về thuật toán, cách áp dụng và thực hành từ đó rút ra kinh nghiệm cho bản thân để áp dụng vào đồ án.

**Trí**

### If you had more time, what would you do?
- Nếu có thêm thời gian, nhóm có thể sẽ tìm hiểu thêm cách tối ưu song song hoá để đạt được thời gian nhanh nhất và kết quả chính xác nhất.

## 6. References